In [1]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import praw
from psaw import PushshiftAPI

api = PushshiftAPI()

import datetime as dt

start_epoch=int(dt.datetime(2021,6,20).timestamp())

list1=list(api.search_submissions(after=start_epoch,
                            subreddit='worldnews',
                            filter=['title']))

/opt/anaconda3/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/anaconda3/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [3]:
new1=[None]*len(list1)

In [4]:
for i in range(0,len(list1)):
    new1[i]=list1[i][3]['title']

s=new1
new=s

In [5]:
for i in range(0,len(s)):
    txt=s[i]
    s[i]=s[i].replace('%','percent')

In [6]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


for i in range(0,len(s)):
    text = s[i]
    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    new[i]=' '.join(tokens_without_sw)

import nltk
nltk.download('averaged_perceptron_tagger')
for i in range(0,len(s)):   
    sentence = new[i]
    tagged_sentence = nltk.tag.pos_tag(sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'JJ' and tag != 'JJR' and tag != 'JJS' and tag != 'RB' and tag != 'RBR' and tag != 'RBS']
    new1[i]=' '.join(edited_sentence)
    
   

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristanmantle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tristanmantle/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
new1=list(filter(lambda a: a != '', new1))

In [8]:
 

import nltk
from pycorenlp import *
import collections

l = [None] * (len(new1))

for i in range(0,(len(new1))):
    nlp=StanfordCoreNLP('http://localhost:9000/')
    x=new1[i]
    output = nlp.annotate(x, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie","triple.strict":"true",
                                     "outputFormat": "json",
                                        "openie.max_entailments_per_clause":"1"})
    
    result = [output["sentences"][0]["openie"] for item in output]
    if len(result[0])>0:
        z=len(result[0])
        relationSent=result[0][z-1]['relation'],result[0][z-1]['subject'],result[0][z-1]['object']
        l[i]=list(relationSent)
    else:
        l[i]=[None,None,None]
       

In [9]:
l.reverse()  
df = pd.DataFrame(l, columns =['P', 'S', 'O'], dtype = float)

In [10]:
df1 = df.replace(to_replace='None', value=np.nan).dropna()
df1.shape[0]/df.shape[0]



0.4819107929515419

In [11]:
triples=df1.values

In [12]:
import math
train=df1[:math.floor(0.75*len(df1))]
valid=df1[math.floor(0.75*len(df1)):math.floor(0.90*len(df1))]
test=df1[math.floor(0.9*len(df1)):]

In [13]:
train_triples=train.values
valid_triples=valid.values
test_triples=test.values

In [14]:
all_data=df1.values

In [15]:
all_data

array([['thousands', 'Ministers eye technology', 'NHS staffing hours'],
       ['cut', 'Rising inflation', 'household incomes'],
       ['hit', 'UK aviation', 'travel rules'],
       ...,
       ['decimates', 'wildfire U.S.', 'Northern California town'],
       ['injuring', 'Man', '10 Tokyo train'],
       ['mark', 'googul Kirk Gibson # Hosting # Golf outing fight',
        'googul.xyz']], dtype=object)

In [16]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

urls=[]

for z in range(0,len(train_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = train_triples[z][2]
        CONFIDENCE = '0.9'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        urls = all_urls + urls

    except:
        
        pass # doing nothing on exception
    

In [20]:
urls = list(dict.fromkeys(urls))
test_urls=urls

In [21]:

    columns = ['S','P', 'O']
    data = []
    whole_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(test_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + test_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/ontology/country"]
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/property/country"]
                
                
                
                frames=[whole_df,extra_df]

                whole_df=pd.concat(frames)

/opt/anaconda3/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:979: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn("Sending Accept header '*/*' because unexpected returned format '%s' in a '%s' SPARQL query form" % (self.returnFormat, self.queryType), RuntimeWarning)


In [22]:
test=whole_df.groupby('P').count()
test.sort_values(by=['O'])

,S,O
P,,
http://dbpedia.org/ontology/Automobile/wheelbase,1,1
http://dbpedia.org/property/logo2Alt,1,1
http://dbpedia.org/property/logo2Caption,1,1
http://dbpedia.org/property/logoImage,1,1
http://dbpedia.org/property/logoPic,1,1
...,...,...
http://dbpedia.org/property/subdivisionName,127333,127333
http://dbpedia.org/property/location,148734,148734
http://www.w3.org/2002/07/owl#sameAs,154101,154101


In [25]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

valid_urls=[]

for z in range(0,len(valid_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = valid_triples[z][2]
        CONFIDENCE = '0.9'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        valid_urls = all_urls + valid_urls

    except:
        
        pass # doing nothing on exception

In [27]:
    valid_urls = list(dict.fromkeys(valid_urls))
    columns = ['S','P', 'O']
    data = []
    valid_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(valid_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + valid_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/ontology/country"]
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/property/country"]
                
                extra_df['O'] = extra_df['O'].astype(str)
                extra_df=extra_df[extra_df['S'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['P'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['O'].str.contains("http://dbpedia.org/",na=False)]
                
                frames=[valid_df,extra_df]

                valid_df=pd.concat(frames)
                

In [28]:
final_df=pd.concat([train, whole_df], sort=False)
valid_final_df=pd.concat([valid, valid_df], sort=False)

In [30]:
final_df["S"] = final_df["S"].astype(str)
final_df["P"] = final_df["P"].astype(str)
final_df["O"] = final_df["O"].astype(str)

valid_final_df["S"] = valid_final_df["S"].astype(str)
valid_final_df["P"] = valid_final_df["P"].astype(str)
valid_final_df["O"] = valid_final_df["O"].astype(str)

In [36]:

#test2=valid_final_df.groupby('P').count()
#test2=test2.sort_values(by=['O'])
#first_v=test2.iloc[:, 0]
#relations_valid=first_v.index.values.tolist() , 
#list(set(relations_valid) - set(relations_train))

In [37]:
final_df = final_df[['S','P','O']]
valid_final_df = valid_final_df[['S','P','O']]

In [61]:
remove_rel=final_df.groupby('P').count()
remove_rel=remove_rel.sort_values(by=['O'])

In [ ]:
single_rel=remove_rel.loc[remove_rel['S'] < 100]
single_rel=single_rel.index.values.tolist()

final_df=final_df[~final_df['P'].isin(single_rel)]
final_df

In [250]:
#remove relations in validation set which do not exist in training set

test1=final_df.groupby('P').count()
test1=test1.sort_values(by=['O'])
first_c=test1.iloc[:, 0]
relations_train=first_c.index.values.tolist()
valid_final_df=valid_final_df[valid_final_df['P'].isin(relations_train)]

In [251]:
test2=final_df.groupby('S').count()
test2=test2.sort_values(by=['O'])
first_c_s=test2.iloc[:, 0]
subject_train=first_c_s.index.values.tolist()
valid_final_df=valid_final_df[valid_final_df['S'].isin(subject_train)]

In [252]:
test3=final_df.groupby('O').count()
test3=test3.sort_values(by=['S'])
first_c_o=test3.iloc[:, 0]
object_train=first_c_o.index.values.tolist()

In [77]:
import ampligraph
ampligraph.latent_features.set_entity_threshold(1542585)

In [78]:
import numpy as np

from ampligraph.latent_features import TransE

train_all_triple=final_df.values
valid_all_triple=valid_final_df.values

model = TransE(batches_count=64, seed=0, epochs=40, k=100, eta=40,optimizer='adam', optimizer_params={'lr':0.0001},loss='pairwise', verbose=True)

model.fit(train_all_triple) #, early_stopping=True, early_stopping_params={'x_valid':valid_all_triple,'burn_in':100}

Average TransE Loss:   0.019796: 100%|██████████| 40/40 [2:44:57<00:00, 247.45s/epoch]  


In [80]:
example_name = 'helloworld.pkl'
ampligraph.utils.save_model(model, model_name_path = example_name)

In [82]:
final_df.to_pickle("final_df.pkl")
valid_final_df.to_pickle("valid_final_df.pkl")

<font size="7">Daily Headline Data</font>

In [106]:
reddit = praw.Reddit(
  client_id='IKnXnWxiUuXhKA',
                     client_secret='wvvyNqFcSHiTLOgfTGcI_FIsaYIyew',
                     user_agent='tm_quant'
)
api = PushshiftAPI(reddit)

start_epoch = int(dt.datetime(2021, 8, 8).timestamp())
end_epoch = int(dt.datetime(2021, 8, 9).timestamp())

gen = api.search_submissions(
  subreddit = 'worldnews',
  filter = ['id'],
  after = start_epoch,
  before = end_epoch
)

results = list(gen)


titles=results
for i in range(0,len(titles)):
    titles[i]=titles[i].title

reddit = praw.Reddit(
  client_id='IKnXnWxiUuXhKA',
                     client_secret='wvvyNqFcSHiTLOgfTGcI_FIsaYIyew',
                     user_agent='tm_quant'
)
api = PushshiftAPI(reddit)

start_epoch = int(dt.datetime(2021, 8, 8).timestamp())
end_epoch = int(dt.datetime(2021, 8, 9).timestamp())

gen = api.search_submissions(
  subreddit = 'worldnews',
  filter = ['id'],
  after = start_epoch,
  before = end_epoch
)

results = list(gen)

scores=results
for i in range(0,len(titles)):
    scores[i]=scores[i].score


In [96]:
#x=len(scores)
#y=len(titles)
#scores=(scores[(x-len(titles)):x])

In [107]:
todays_news_dict = {'Title':titles,'Score':scores}
todays_news = pd.DataFrame(todays_news_dict)

todays_news=todays_news.sort_values(by=['Score'], ascending=False)

todays_news=todays_news.Title.values.tolist()

top_news=todays_news[0:50]

In [109]:
headlines=top_news
s = top_news
new=s
new1=s

for i in range(0,len(s)):
    txt=s[i]
    s[i]=s[i].replace('%','percent')




from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


for i in range(0,len(s)):
    text = s[i]
    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    new[i]=' '.join(tokens_without_sw)

import nltk
nltk.download('averaged_perceptron_tagger')
for i in range(0,len(s)):   
    sentence = new[i]
    tagged_sentence = nltk.tag.pos_tag(sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'JJ' and tag != 'JJR' and tag != 'JJS' and tag != 'RB' and tag != 'RBR' and tag != 'RBS']
    new1[i]=' '.join(edited_sentence)

import nltk
from pycorenlp import *
import collections

l = [None] * (len(headlines))

for i in range(0,len(headlines)):
    nlp=StanfordCoreNLP('http://localhost:9000/')
    x=new1[i]
    output = nlp.annotate(x, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie","triple.strict":"true",
                                     "outputFormat": "json",
                                        "openie.max_entailments_per_clause":"1"})
    
    result = [output["sentences"][0]["openie"] for item in output]
    if len(result[0])>0:
        z=len(result[0])
        relationSent=result[0][z-1]['relation'],result[0][z-1]['subject'],result[0][z-1]['object']
        l[i]=list(relationSent)
    else:
        l[i]=[None,None,None]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristanmantle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tristanmantle/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [110]:
day_df = pd.DataFrame(l, columns =['P', 'S', 'O'], dtype = float)
day_df = day_df.replace(to_replace='None', value=np.nan).dropna()

In [111]:
day_triples=day_df.values

In [118]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

day_urls=[]

for z in range(0,len(day_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = day_triples[z][2]
        CONFIDENCE = '0.7'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        day_urls = all_urls + day_urls

    except:
        
        pass # doing nothing on exception

In [154]:
day_urls = list(dict.fromkeys(day_urls))

In [189]:
    columns = ['S','P', 'O']
    data = []
    whole_day_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(day_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + day_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/ontology/country"]
                extra_df=extra_df.loc[extra_df['P'] != "http://dbpedia.org/property/country"]
                
                extra_df['O'] = extra_df['O'].astype(str)
                extra_df=extra_df[extra_df['S'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['P'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['O'].str.contains("http://dbpedia.org/",na=False)]
                
                
                
                frames=[whole_day_df,extra_df]

                whole_day_df=pd.concat(frames)

In [255]:
whole_day_df=whole_day_df[whole_day_df['P'].isin(relations_train)]
whole_day_df=whole_day_df[whole_day_df['S'].isin(subject_train)]
whole_day_df=whole_day_df[whole_day_df['O'].isin(object_train)]

In [192]:
##check=whole_day_df.groupby('O').count()
#check=check.sort_values(by=['S'])
#check_s=check.iloc[:, 0]
#relations_day=check_s.index.values.tolist()
#list(set(relations_day) - set(object_train))

In [256]:
day_triples=whole_day_df.values

In [257]:
scores = model.predict(day_triples)

In [265]:
import statistics
statistics.mean(scores)

-2.6190782